# 제조 공정 내 가스 혼합물의 개별 가스 성분 분류

## 데이터 소개 및 분석 프로세스 수립

### 데이터 수집  
데이터 소스: https://archive.ics.uci.edu/static/public/270/gas+sensor+array+drift+dataset+at+different+concentrations.zip

< 데이터 >  

|GAS|1-128|  
|--|--|  
|혼합가스종류|금속 산화물 가스 센서 계측값들|

### 데이터 전처리
- Target Feature 생성
- 상관성 분석

### 모델 생성  
- PCA 적용
- Random Forest, Ada Boost, Bagging with KNN, Decision Tree, Voting Ensemble

### 모델 평가 및 의사결정
- Feature Selection
- 재모델링